Samler data for temperatur, lufttrykk og nedbør i en csv fil:

In [13]:
import pandas as pd

# Leser inn de 3 filene for vær verdier
df_gj = pd.read_csv("frost_air_temperature_daglig_gjennomsnitt.csv")
df_trykk = pd.read_csv("frost_meanair_pressure_at_sea_level_P1D_jul-des_2024.csv")
df_nedbør = pd.read_csv("frost_sumprecipitation_amount_P1D_jul-des_2024.csv")

# Slår sammen filene til en fil, basert på dato
samlet = df_gj \
    .merge(df_trykk, on="Dato", how="left") \
    .merge(df_nedbør, on="Dato", how="left")

samlet.rename(columns={
    "Temperatur (°C)": "Temperatur (°C) daglig snitt"
}, inplace=True)

# Oppretter ny fil for samlet verdier
samlet.to_csv("Sammenslaaing_gjsnitt.csv", index=False)

print(samlet.head())


         Dato  Temperatur (°C) daglig snitt  Lufttrykk (hPa)  Nedbør (mm)
0  2024-07-01                     10.393750           1003.0          8.0
1  2024-07-02                     10.177778           1002.9          3.6
2  2024-07-03                     11.973611           1000.5          2.0
3  2024-07-04                     15.785417            993.2          0.1
4  2024-07-05                     12.113889            991.8          7.7


Identifisere og håndtere manglende verdier i datasettet:

In [5]:
samlet.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 0 to 182
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Dato                          183 non-null    object 
 1   Temperatur (°C) daglig snitt  183 non-null    float64
 2   Lufttrykk (hPa)               183 non-null    float64
 3   Nedbør (mm)                   183 non-null    float64
dtypes: float64(3), object(1)
memory usage: 7.1+ KB


In [6]:
print(samlet.isna().sum())

Dato                            0
Temperatur (°C) daglig snitt    0
Lufttrykk (hPa)                 0
Nedbør (mm)                     0
dtype: int64


Kode for å lokalisere "NaN" verdier:

In [1]:
import pandas as pd

df_sammenslaaing = pd.read_csv("Sammenslaaing_gjsnitt.csv")

df_sammenslaaing.loc[df_sammenslaaing.isnull().any(axis=1)]


,Dato,Temperatur (°C) daglig snitt,Lufttrykk (hPa),Nedbør (mm)
5,2024-07-06,12.230556,NaN,3.2
33,2024-08-03,NaN,1006.8,0.0


Lager ny csv for oppdaterte (riktige) verdier:

In [2]:
df_sammenslaaing_UtenFeil = df_sammenslaaing.copy()

Kode for å rette opp i eventuelle negative nedbør og/eller lufttrykk-verdier:

In [8]:
df_sammenslaaing_UtenFeil['Nedbør (mm)'] = [abs(x) for x in df_sammenslaaing_UtenFeil['Nedbør (mm)']]
df_sammenslaaing_UtenFeil.to_csv("Sammenslaaing_uten_feil.csv")

df_sammenslaaing_UtenFeil['Lufttrykk (hPa)'] = [abs(x) for x in df_sammenslaaing_UtenFeil['Lufttrykk (hPa)']]
df_sammenslaaing_UtenFeil.to_csv("Sammenslaaing_uten_feil.csv")

Gjør at datoene blir i datetime-format så ikke det er noen skrivefeil i datoene henta:

In [24]:
# Gjør at 'Dato' er i datetime-format
df_sammenslaaing_UtenFeil['Dato'] = pd.to_datetime(df_sammenslaaing_UtenFeil['Dato'])

# Start fra første dato og generer resten
start_dato = df_sammenslaaing_UtenFeil.loc[0, 'Dato']
riktige_datoer = [start_dato + pd.Timedelta(days=i) for i in range(len(df_sammenslaaing_UtenFeil))]

# Sett inn de riktige datoene
df_sammenslaaing_UtenFeil['Dato'] = riktige_datoer


print(df_sammenslaaing_UtenFeil.head())

        Dato  Temperatur (°C) daglig snitt  Lufttrykk (hPa)  Nedbør (mm)
0 2024-07-01                     10.393750           1003.0          8.0
1 2024-07-02                     10.177778           1002.9          3.6
2 2024-07-03                     11.973611           1000.5          2.0
3 2024-07-04                     15.785417            993.2          0.1
4 2024-07-05                     12.113889            991.8          7.7


Kode for å rette opp i ussansynlige temperaturverdier, samt itterere for å sjekke om en temperatur ikke gir verdier(NaN). Bytter ut disse mistenkelige verdiene med ett gjennomsnitt fra dagen før opg dagen etter.

In [ ]:
import pandas as pd

# Les inn CSV-filen
df = pd.read_csv("Sammenslaaing_gjsnitt.csv")

# Gå gjennom temperaturkolonnen med indekser
for i in range(1, len(df) - 1):  
    temp = df.loc[i, "Temperatur (°C) daglig snitt"]

    # Hvis verdien er mistenkelig eller tom
    if pd.isna(temp) or temp <= -50 or temp >= 50:

        # Gjennomsnitt av dagen før og dagen etter
        temp_forrige = df.loc[i - 1, "Temperatur (°C) daglig snitt"]
        temp_etter = df.loc[i + 1, "Temperatur (°C) daglig snitt"]
        gjennomsnitt = (temp_forrige + temp_etter) / 2

        # Erstatt verdien
        df.loc[i, "Temperatur (°C) daglig snitt"] = gjennomsnitt

# Skriv til filen uten feil
df.to_csv("Sammenslaaing_uten_feil.csv", index=False)

Kode for å teste om lufttrykk-verdien er tom:

In [ ]:
import pandas as pd

# Les inn CSV-filen
df = pd.read_csv("Sammenslaaing_gjsnitt.csv")

# Gå gjennom temperaturkolonnen med indekser
for i in range(1, len(df) - 1):  
    trykk = df.loc[i, "Lufttrykk (hPa)"]

    # Hvis verdien er mistenkelig eller tom
    if pd.isna(trykk):

        # Gjennomsnitt av dagen før og dagen etter
        trykk_forrige = df.loc[i - 1, "Lufttrykk (hPa)"]
        trykk_etter = df.loc[i + 1, "Lufttrykk (hPa)"]
        gjennomsnitt = (trykk_forrige + trykk_etter) / 2

        # Erstatt verdien
        df.loc[i, "Lufttrykk (hPa)"] = gjennomsnitt

# Skriv til filen uten feil
df.to_csv("Sammenslaaing_uten_feil.csv", index=False)